# Investigate the deeplabcut infrastructure modules
This is a notebook where I am going through the code in DLC to understand it :-)

In [1]:
import deeplabcut
from pathlib import Path
import os.path

## Test the autotune setting in TF 1.10.0
Pending

In [2]:
import tensorflow as tf
print(tf.__version__)

1.10.0


In [4]:
# config_path = "/home/antortjim/MEGA/FlySleepLab/DeepLabCut/examples/Arena-El-Sayed-2019-02-03/config.yaml"
# poseconfigfile=Path("/home/antortjim/MEGA/FlySleepLab/DeepLabCut/examples/Arena-El-Sayed-2019-02-03/dlc-models/iteration-0/ArenaFeb3-trainset95shuffle1/train/pose_cfg.yaml")
DLC_path=!pwd
DLC_path=DLC_path[0]

# DLC_path = "/home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC"

main_path=os.path.join(DLC_path, "examples/Reaching-Mackenzie-2018-08-30")
config_path = os.path.join(main_path, "config.yaml")
poseconfigfile=Path(os.path.join(main_path, "dlc-models/iteration-0/ReachingAug30-trainset95shuffle1/train/pose_cfg.yaml"))
config_yaml = poseconfigfile
print(config_yaml)

/home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Reaching-Mackenzie-2018-08-30/dlc-models/iteration-0/ReachingAug30-trainset95shuffle1/train/pose_cfg.yaml


# The `train()` function in `pose_estimation_tensorflow/train.py`

deeplabcut.train_network() is just a CLI interface for the most common call to train(). Let us go deep into its code in order to understand it!!!

In [5]:
shuffle=1
trainingsetindex=0
gputouse=None
max_snapshots_to_keep=5
max_to_keep=max_snapshots_to_keep
utotune=False
displayiters=100
display_iters=displayiters
# saveiters=1000
saveiters=100
save_iters=saveiters
# maxiters=200000
maxiters=100

max_iter=maxiters

import logging, os
start_path=os.getcwd()
os.chdir(str(Path(config_yaml).parents[0])) #switch to folder of config_yaml (for logging)
from deeplabcut.pose_estimation_tensorflow.util.logging import setup_logging
# def setup_logging():
#     FORMAT = '%(asctime)-15s %(message)s'
#     logging.basicConfig(filename=os.path.join('log.txt'), filemode='w',
#                         datefmt='%Y-%m-%d %H:%M:%S',
#                         level=logging.INFO, format=FORMAT)

#     console = logging.StreamHandler()
#     console.setLevel(logging.INFO)
#     logging.getLogger('').addHandler(console)

setup_logging()

## How is cfg created in train() @ pose_estimation_tensorflow/train.py?

In [6]:
cfg = deeplabcut.pose_estimation_tensorflow.config.load_config(poseconfigfile)

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['Hand', 'Finger1', 'Finger2', 'Joystick'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_ReachingAug30/Reaching_Mackenzie95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/antortjim/anaconda3/envs/DLC/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_ReachingAug30/Documentation_data-Reaching_95shuffle1.pickle',
 'minsize': 100

1. cfg is the output of `pose_estimation_tensorflow.config.load_config(path/poseconfigfile)`
2. `load_config()` is a call to `pose_estimation_tensorflow.config.cfg_from_file()` with the only argument it has set to: `"pose_cfg.yaml"`
3. `cfg_from_file`:
  * reads the pose_config.yaml file into an EasyDict
  * sets the snapshot_prefix to a default (depends on the trainpath, not default)
  * merges the pose_config with the default configurations (extracted via `cfg = auxiliaryfunctions.read_config(config)`)
  * merging is done via `pose_estimation_tensorflow.config._merge_a_into_b(a, b)`, a recursive function that calls itself!

4. Again in train(), the batch_size is set to a default of 1

## What is the dataset object?

In [13]:
from deeplabcut.pose_estimation_tensorflow.dataset.factory import create as create_dataset
dataset = create_dataset(cfg)
print("The current dataset has {} images".format(dataset.num_images))

i=0
dataset_array=dataset.raw_data["dataset"]
sample=dataset_array[0,i]
# len(sample)
# print(sample[0][0]) # path
# sample[1][0] # size [channel, h, w]
# joints = sample[2][0][0] # features [[id, x, y],
#                          #           [id, x, y]]

The current dataset has 18 images


Instantiates a `PoseDataset` object with the configuration supplied in cfg
It will have
1. Metadata attributes describing the dataset
2. A raw_data attribute with metadata and another attribute called dataset. This contains metadata about the images!
  * It's an array of size (1, num_images)
  * Each ith element is a list like object (np.void) for the ith image
  * [0][0] gives the path to the image
  * [1][0] is the size (channels, h, w)
  * [2][0] is the features (joints) in an array of shape nx3 where n is the number of features (feature_id, x, y)


## What is batch_spec?

In [9]:
from deeplabcut.pose_estimation_tensorflow.nnet.pose_net import get_batch_spec

from deeplabcut.pose_estimation_tensorflow.dataset.pose_dataset import Batch
# class Batch(Enum):
#     inputs = 0
#     part_score_targets = 1
#     part_score_weights = 2
#     locref_targets = 3
#     locref_mask = 4
#     data_item = 5

    
# def get_batch_spec(cfg):
#     num_joints = cfg.num_joints
#     batch_size = cfg.batch_size
#     return {
#         Batch.inputs: [batch_size, None, None, 3],
#         Batch.part_score_targets: [batch_size, None, None, num_joints],
#         Batch.part_score_weights: [batch_size, None, None, num_joints],
#         Batch.locref_targets: [batch_size, None, None, num_joints * 2],
#         Batch.locref_mask: [batch_size, None, None, num_joints * 2]
#     }

batch_spec = get_batch_spec(cfg)

Consists of a dictionary with 5 entries, each storing a list of length 4.
The keys in this dictionary are Enum objects defined in:

`from deeplabcut.pose_estimation_tensorflow.dataset.pose_dataset import Batch`

In [10]:
batch_spec

{<Batch.inputs: 0>: [1, None, None, 3],
 <Batch.part_score_targets: 1>: [1, None, None, 4],
 <Batch.part_score_weights: 2>: [1, None, None, 4],
 <Batch.locref_targets: 3>: [1, None, None, 8],
 <Batch.locref_mask: 4>: [1, None, None, 8]}

## What is batch, enqueue_op and placeholders (output of setup_preloading)

In [10]:
from deeplabcut.pose_estimation_tensorflow.train import setup_preloading
# def setup_preloading(batch_spec):
#     placeholders = {name: tf.placeholder(tf.float32, shape=spec) for (name, spec) in batch_spec.items()}
#     names = placeholders.keys()
#     placeholders_list = list(placeholders.values())

#     QUEUE_SIZE = 20

#     q = tf.FIFOQueue(QUEUE_SIZE, [tf.float32]*len(batch_spec))
#     enqueue_op = q.enqueue(placeholders_list)
#     batch_list = q.dequeue()

#     batch = {}
#     for idx, name in enumerate(names):
#         batch[name] = batch_list[idx]
#         batch[name].set_shape(batch_spec[name])
#     return batch, enqueue_op, placeholders
batch, enqueue_op, placeholders = setup_preloading(batch_spec)

**batch** is a dictionary analogous to batch_spec that contains tf.tensors of class `tf.Tensor 'fifo_queue_1_Dequeue:i'`. Length = to the lists in batch_spec. float32

In [11]:
batch

{<Batch.inputs: 0>: <tf.Tensor 'fifo_queue_Dequeue:0' shape=(1, ?, ?, 3) dtype=float32>,
 <Batch.part_score_targets: 1>: <tf.Tensor 'fifo_queue_Dequeue:1' shape=(1, ?, ?, 4) dtype=float32>,
 <Batch.part_score_weights: 2>: <tf.Tensor 'fifo_queue_Dequeue:2' shape=(1, ?, ?, 4) dtype=float32>,
 <Batch.locref_targets: 3>: <tf.Tensor 'fifo_queue_Dequeue:3' shape=(1, ?, ?, 8) dtype=float32>,
 <Batch.locref_mask: 4>: <tf.Tensor 'fifo_queue_Dequeue:4' shape=(1, ?, ?, 8) dtype=float32>}

**enqueue_op** is a tf.Operation object:
`<tf.Operation 'fifo_queue_1_enqueue' type=QueueEnqueueV2>`

**placeholders** is a dictionary like **batch** storing tensors of class a `tf.Tensor 'Placeholder_5:0'`. Length the same and float 32

In [12]:
placeholders

{<Batch.inputs: 0>: <tf.Tensor 'Placeholder:0' shape=(1, ?, ?, 3) dtype=float32>,
 <Batch.part_score_targets: 1>: <tf.Tensor 'Placeholder_1:0' shape=(1, ?, ?, 4) dtype=float32>,
 <Batch.part_score_weights: 2>: <tf.Tensor 'Placeholder_2:0' shape=(1, ?, ?, 4) dtype=float32>,
 <Batch.locref_targets: 3>: <tf.Tensor 'Placeholder_3:0' shape=(1, ?, ?, 8) dtype=float32>,
 <Batch.locref_mask: 4>: <tf.Tensor 'Placeholder_4:0' shape=(1, ?, ?, 8) dtype=float32>}

To sum up: batch, batch_spec and placeholders are dictionaries with 5 entries. Every key is a Batch object (Enum class). The 5 keys are shared by all 3 dictionaries.
* batch stores **TF queue_Dequeue tensors**
* batch_spec stores **Python lists**
* placeholders stores **TF Placeholder tensors**

## What are the 5 keys??

## What is losses?

Losses is a dict with three different losses, each a tensor
1. **part_loss**: `<tf.Tensor 'sigmoid_cross_entropy_loss/value:0' shape=() dtype=float32>`
2. **locref_loss**: `<tf.Tensor 'mul:0' shape=() dtype=float32>`
3. **total_loss**: `<tf.Tensor 'add:0' shape=() dtype=float32>`

It's the output of the `train` method in the PoseNet class.
  * `PoseNet.train(batch)
  
Their value is tracked with the `summary.scalars`

In [17]:
from deeplabcut.pose_estimation_tensorflow.dataset.pose_dataset import Batch
print(batch[Batch.part_score_weights])
print(batch.keys())

Tensor("fifo_queue_Dequeue:2", shape=(1, ?, ?, 4), dtype=float32)
dict_keys([<Batch.inputs: 0>, <Batch.part_score_targets: 1>, <Batch.part_score_weights: 2>, <Batch.locref_targets: 3>, <Batch.locref_mask: 4>])


In [ ]:
from deeplabcut.pose_estimation_tensorflow.nnet.net_factory import pose_net
losses = pose_net(cfg).train(batch)

## Looking deeper into the pose_net(cfg).train(batch) statement
This statement does the following:

* Instantiate an object of class PoseNet taking the cfg file as input. cfg is assigned to self
* Then, the train method in the object is called with batch as input

**The train() method** runs the get_net() method with the value under Batch.input key in the batch dictionary. This gives us heads, which is a dictionary with 2 keys, `part_pred` and `locref` (1). The values are `slim.conv2d_transpose` objects, the output of `prediction_layer`. This objects store a 

1. `part_pred_interm` is a third key available only if `location_refinement` is true

In [18]:
    def train(self, batch):
        cfg = self.cfg

        heads = self.get_net(batch[Batch.inputs])

        weigh_part_predictions = cfg.weigh_part_predictions
        part_score_weights = batch[Batch.part_score_weights] if weigh_part_predictions else 1.0

        def add_part_loss(pred_layer):
            return tf.losses.sigmoid_cross_entropy(batch[Batch.part_score_targets],
                                                   heads[pred_layer],
                                                   part_score_weights)

        loss = {}
        loss['part_loss'] = add_part_loss('part_pred')
        total_loss = loss['part_loss']
        if cfg.intermediate_supervision:
            loss['part_loss_interm'] = add_part_loss('part_pred_interm')
            total_loss = total_loss + loss['part_loss_interm']

        if cfg.location_refinement:
            locref_pred = heads['locref']
            locref_targets = batch[Batch.locref_targets]
            locref_weights = batch[Batch.locref_mask]

            loss_func = losses.huber_loss if cfg.locref_huber_loss else tf.losses.mean_squared_error
            loss['locref_loss'] = cfg.locref_loss_weight * loss_func(locref_targets, locref_pred, locref_weights)
            total_loss = total_loss + loss['locref_loss']

        # loss['total_loss'] = slim.losses.get_total_loss(add_regularization_losses=params.regularize)
        loss['total_loss'] = total_loss
        return loss

In [ ]:
 This is summarised by:  * 


In [13]:
def prediction_layer(cfg, input, name, num_outputs):
    with slim.arg_scope([slim.conv2d, slim.conv2d_transpose], padding='SAME',
                        activation_fn=None, normalizer_fn=None,
                        weights_regularizer=slim.l2_regularizer(cfg.weight_decay)):
        with tf.variable_scope(name):
            pred = slim.conv2d_transpose(input, num_outputs,
                                         kernel_size=[3, 3], stride=2,
                                         scope='block4')
            return pred

In [14]:
def prediction_layers(self, features, end_points, reuse=None):
    cfg = self.cfg

    num_layers = re.findall("resnet_([0-9]*)", cfg.net_type)[0]
    layer_name = 'resnet_v1_{}'.format(num_layers) + '/block{}/unit_{}/bottleneck_v1'

    out = {}
    with tf.variable_scope('pose', reuse=reuse):
        out['part_pred'] = prediction_layer(cfg, features, 'part_pred',
                                            cfg.num_joints)
        if cfg.location_refinement:
            out['locref'] = prediction_layer(cfg, features, 'locref_pred',
                                             cfg.num_joints * 2)
        if cfg.intermediate_supervision:
            interm_name = layer_name.format(3, cfg.intermediate_supervision_layer)
            block_interm_out = end_points[interm_name]
            out['part_pred_interm'] = prediction_layer(cfg, block_interm_out,
                                                       'intermediate_supervision',
                                                       cfg.num_joints)
    return out


In [15]:
from tensorflow.contrib.slim.nets import resnet_v1
from deeplabcut.pose_estimation_tensorflow.nnet import losses


# net_funcs = {'resnet_50': resnet_v1.resnet_v1_50,
#              'resnet_101': resnet_v1.resnet_v1_101}
# class PoseNet:
#     def __init__(self, cfg):
#         self.cfg = cfg
#
#     def extract_features(self, inputs):
#         net_fun = net_funcs[self.cfg.net_type]

#         mean = tf.constant(self.cfg.mean_pixel,
#                            dtype=tf.float32, shape=[1, 1, 1, 3], name='img_mean')
#         im_centered = inputs - mean

#         # The next part of the code depends upon which tensorflow version you have.
#         vers = tf.__version__
#         vers = vers.split(".") #Updated based on https://github.com/AlexEMG/DeepLabCut/issues/44
#         if int(vers[0])==1 and int(vers[1])<4: #check if lower than version 1.4.
#             with slim.arg_scope(resnet_v1.resnet_arg_scope(False)):
#                 net, end_points = net_fun(im_centered,
#                                           global_pool=False, output_stride=16)
#         else:
#             with slim.arg_scope(resnet_v1.resnet_arg_scope()):
#                 net, end_points = net_fun(im_centered,
#                                           global_pool=False, output_stride=16,is_training=False)

#         return net,end_points

#
#     def get_net(self, inputs):
#         net, end_points = self.extract_features(inputs)
#         return self.prediction_layers(net, end_points)
#
#


    


In [16]:
for k, t in losses.items():
    tf.summary.scalar(k, t)
merged_summaries = tf.summary.merge_all()
total_loss = losses["total_loss"]

In [17]:
## Variables to restore?

In [18]:
import tensorflow.contrib.slim as slim
variables_to_restore = slim.get_variables_to_restore(include=["resnet_v1"])
restorer = tf.train.Saver(variables_to_restore)
saver = tf.train.Saver(max_to_keep=max_to_keep)    

Huge amount of variables extracted from the pretrained ResNet50 print(len(variables_to_restore))
TODO: Google restorers and savers!

In [19]:
saver

In [20]:
import threading
from deeplabcut.pose_estimation_tensorflow.train import load_and_enqueue
# def load_and_enqueue(sess, enqueue_op, coord, dataset, placeholders):
#     while not coord.should_stop():
#         batch_np = dataset.next_batch()
#         food = {pl: batch_np[name] for (name, pl) in placeholders.items()}
#         sess.run(enqueue_op, feed_dict=food)

        
from deeplabcut.pose_estimation_tensorflow.train import start_preloading
# def start_preloading(sess, enqueue_op, dataset, placeholders):
#     coord = tf.train.Coordinator()

#     t = threading.Thread(target=load_and_enqueue,
#                          args=(sess, enqueue_op, coord, dataset, placeholders))
#     t.start()

#     return coord, t

In [21]:
sess = tf.Session()
coord, thread = start_preloading(sess, enqueue_op, dataset, placeholders)

In [22]:
# coord

In [23]:
# thread

In [24]:
from deeplabcut.pose_estimation_tensorflow.train import get_optimizer
# def get_optimizer(loss_op, cfg):
#     learning_rate = tf.placeholder(tf.float32, shape=[])

#     if cfg.optimizer == "sgd":
#         optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
#     elif cfg.optimizer == "adam":
#         optimizer = tf.train.AdamOptimizer(cfg.adam_lr)
#     else:
#         raise ValueError('unknown optimizer {}'.format(cfg.optimizer))
#     train_op = slim.learning.create_train_op(loss_op, optimizer)

#     return learning_rate, train_op

In [25]:
train_writer = tf.summary.FileWriter(cfg.log_dir, sess.graph)
learning_rate, train_op = get_optimizer(total_loss, cfg)

In [26]:
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

In [27]:
# Restore variables from disk.
restorer.restore(sess, cfg.init_weights)
if maxiters==None:
    max_iter = int(cfg.multi_step[-1][1])
else:
    max_iter = min(int(cfg.multi_step[-1][1]),int(maxiters))
    #display_iters = max(1,int(displayiters))
    print("Max_iters overwritten as",max_iter)

if displayiters==None:
    display_iters = max(1,int(cfg.display_iters))
else:
    display_iters = max(1,int(displayiters))
    print("Display_iters overwritten as",display_iters)

if saveiters==None:
     save_iters=max(1,int(cfg.save_iters))

else:
    save_iters=max(1,int(saveiters))
    print("Save_iters overwritten as",save_iters)

INFO:tensorflow:Restoring parameters from /home/antortjim/anaconda3/envs/DLC/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt


Restoring parameters from /home/antortjim/anaconda3/envs/DLC/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt


Max_iters overwritten as 100
Display_iters overwritten as 100
Save_iters overwritten as 100


In [28]:
from deeplabcut.pose_estimation_tensorflow.train import LearningRate

# class LearningRate(object):
#     def __init__(self, cfg):
#         self.steps = cfg.multi_step
#         self.current_step = 0

#     def get_lr(self, iteration):
#         lr = self.steps[self.current_step][0]
#         if iteration == self.steps[self.current_step][1]:
#             self.current_step += 1

#         return lr

    
cum_loss = 0.0
lr_gen = LearningRate(cfg)
stats_path = Path(config_yaml).with_name('learning_stats.csv')
lrf = open(str(stats_path), 'w')

In [29]:
lr_gen

In [30]:
print("Training parameter:")
print(cfg)
print("Starting training....")
for it in range(max_iter+1):
    print(it)
    current_lr = lr_gen.get_lr(it)
    [_, loss_val, summary] = sess.run([train_op, total_loss, merged_summaries],
                                      feed_dict={learning_rate: current_lr})
    cum_loss += loss_val
    train_writer.add_summary(summary, it)
  
    if it % display_iters == 0 and it>0:
        average_loss = cum_loss / display_iters
        cum_loss = 0.0
        logging.info("iteration: {} loss: {} lr: {}"
                     .format(it, "{0:.4f}".format(average_loss), current_lr))
        lrf.write("{}, {:.5f}, {}\n".format(it, average_loss, current_lr))
        lrf.flush()

    # Save snapshot
    if (it % save_iters == 0 and it != 0) or it == max_iter:
        model_name = cfg.snapshot_prefix
        saver.save(sess, model_name, global_step=it)

Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/home/antortjim/MEGA/FlySleepLab/DeepLabCut/examples/Reaching-Mackenzie-2018-08-30/dlc-models/iteration-0/ReachingAug30-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scoremap_dir': 'test', 'dataset_type': 'default', 'use_gt_segm': False, 'batch_size': 1, 'video': False, 'video_batch': False, 'crop': True, 'cropratio': 0.4, 'minsize': 100, 'leftwidth': 400, 'rightwidth': 400, 'topheight': 400, 'bottomheight': 400, 'all_joints': [[0], [1], [2], [3]], 'all_joi

iteration: 100 loss: 0.0553 lr: 0.005


In [32]:
lrf.close()
sess.close()
coord.request_stop()
coord.join([thread])
#return to original path.
os.chdir(str(start_path))

In [33]:
start_path

'/home/antortjim/MEGA/FlySleepLab/DeepLabCut/examples'